In [9]:
import pandas as pd
from igf_data.igfdb.igfTables import Seqrun, Seqrun_attribute, Platform
from igf_data.igfdb.seqrunadaptor import SeqrunAdaptor

In [10]:
pipe_seed=pd.DataFrame([{'pipeseed_id':'1','seed_id':'1','seed_table':'SEQRUN'},
                        {'pipeseed_id':'2','seed_id':'2','seed_table':'SEQRUN'},
                       ])

In [11]:
pipe_seed

,pipeseed_id,seed_id,seed_table
0,1,1,SEQRUN
1,2,2,SEQRUN


In [12]:
seed_data=pipe_seed[['seed_id','seed_table']]

In [13]:
seed_data

,seed_id,seed_table
0,1,SEQRUN
1,2,SEQRUN


In [12]:
def map_seed_data(x):
    if x.seed_table=='SEQRUN':
        print('its seqrun')
    else:
        print('its not seqrun')

In [15]:
seed_data.apply(lambda x: map_seed_data(x), axis=1)

its seqrun
its seqrun


0    None
1    None
dtype: object

In [2]:
dbparams={'dbhost':'0.0.0.0', \
          'dbuser':'root', \
          'dbpass':'root123', \
          'dbname':'igfdb', \
          'driver':'mysql', \
          'connector':'pymysql'}

In [3]:
sa=SeqrunAdaptor(**dbparams)

In [4]:
sa.start_session()

In [21]:
sa.close_session()

In [18]:
sa_query=sa.session.query(Seqrun,
                          Platform.platform_igf_id,
                          Platform.model_name,
                          Platform.vendor_name,
                          Platform.software_name,
                          Platform.software_version,
                         ).join(Platform)

In [19]:
def map_seed_data(x,sa_query):
    x=pd.Series(x)
    if x.seed_table=='SEQRUN':
        query=sa_query.filter(Seqrun.seqrun_id==x.seed_id)
        data=sa.fetch_records(query)
        return data
    #    print(x)
    else:
        print('its not seqrun')

In [20]:
da=pd.concat([map_seed_data(x=record,sa_query=sa_query) for record in seed_data.to_dict(orient='records')], axis=0)

In [25]:
import datetime

In [37]:
da['date_created']=da['date_created'].astype(str)

In [38]:
da.to_dict(orient='records')

[{'date_created': '2017-09-05 16:00:18',
  'flowcell_id': 'AHJJKTBBXX',
  'model_name': 'HISEQ4000',
  'platform_id': 1,
  'platform_igf_id': 'ILM4K_001',
  'reject_run': 'N',
  'seqrun_id': 1,
  'seqrun_igf_id': '170627_K00345_0012_AHJJKTBBXX',
  'software_name': 'RTA',
  'software_version': 'RTA2',
  'vendor_name': 'ILLUMINA'},
 {'date_created': '2017-09-05 16:00:18',
  'flowcell_id': 'D0V30',
  'model_name': 'HISEQ4000',
  'platform_id': 1,
  'platform_igf_id': 'ILM4K_001',
  'reject_run': 'N',
  'seqrun_id': 2,
  'seqrun_igf_id': '170627_M03291_0071_000000000-D0V30',
  'software_name': 'RTA',
  'software_version': 'RTA2',
  'vendor_name': 'ILLUMINA'}]

In [21]:
da.to_json(orient='records')

'[{"seqrun_id":1,"seqrun_igf_id":"170627_K00345_0012_AHJJKTBBXX","reject_run":"N","date_created":1504627218000,"flowcell_id":"AHJJKTBBXX","platform_id":1,"platform_igf_id":"ILM4K_001","model_name":"HISEQ4000","vendor_name":"ILLUMINA","software_name":"RTA","software_version":"RTA2"},{"seqrun_id":2,"seqrun_igf_id":"170627_M03291_0071_000000000-D0V30","reject_run":"N","date_created":1504627218000,"flowcell_id":"D0V30","platform_id":1,"platform_igf_id":"ILM4K_001","model_name":"HISEQ4000","vendor_name":"ILLUMINA","software_name":"RTA","software_version":"RTA2"}]'

In [124]:
da

,seqrun_id,seqrun_igf_id,flowcell_id,reject_run,platform_igf_id,model_name,vendor_name,software_name,software_version
0,1,170627_K00345_0012_AHJJKTBBXX,AHJJKTBBXX,N,ILM4K_001,HISEQ4000,ILLUMINA,RTA,RTA2
0,2,170627_M03291_0071_000000000-D0V30,D0V30,N,ILM4K_001,HISEQ4000,ILLUMINA,RTA,RTA2


In [135]:
pipe_seed

,pipeseed_id,seed_id,seed_table
0,1,1,SEQRUN
1,2,2,SEQRUN


In [134]:
pipe_seed['seed_id']=pipe_seed['seed_id'].astype(int)

In [138]:
merged_df=pd.merge(pipe_seed,da,how='inner',on=None,left_on=['seed_id'],right_on=['seqrun_id'],left_index=False,right_index=False)

In [139]:
merged_df.to_dict(orient='records')

[{'flowcell_id': 'AHJJKTBBXX',
  'model_name': 'HISEQ4000',
  'pipeseed_id': '1',
  'platform_igf_id': 'ILM4K_001',
  'reject_run': 'N',
  'seed_id': 1,
  'seed_table': 'SEQRUN',
  'seqrun_id': 1,
  'seqrun_igf_id': '170627_K00345_0012_AHJJKTBBXX',
  'software_name': 'RTA',
  'software_version': 'RTA2',
  'vendor_name': 'ILLUMINA'},
 {'flowcell_id': 'D0V30',
  'model_name': 'HISEQ4000',
  'pipeseed_id': '2',
  'platform_igf_id': 'ILM4K_001',
  'reject_run': 'N',
  'seed_id': 2,
  'seed_table': 'SEQRUN',
  'seqrun_id': 2,
  'seqrun_igf_id': '170627_M03291_0071_000000000-D0V30',
  'software_name': 'RTA',
  'software_version': 'RTA2',
  'vendor_name': 'ILLUMINA'}]

In [8]:
list(pipe_seed.columns)

['pipeseed_id', 'seed_id', 'seed_table']

In [32]:
pipe_seed.loc[pipe_seed.seed_id=='1']['seed_table'].values

array(['SEQRUN'], dtype=object)

In [40]:
import pip

In [41]:
pip.main(['install','pysftp'])

  Running setup.py install for pycparser: started
    Running setup.py install for pycparser: finished with status 'done'
  Running setup.py install for pysftp: started
    Running setup.py install for pysftp: finished with status 'done'


You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


0

In [44]:
import pysftp

In [50]:
with pysftp.Connection('orwell.hh.med.ic.ac.uk',username='igf',password='4ZSsWqen',cnopts=pysftp.CnOpts({hostkeys=None})) as sftp:
    sftp.listdir(remotepath='/home/igf/igf_work')

SyntaxError: invalid syntax (<ipython-input-50-15825e1fae4e>, line 1)